In [168]:
from pyspark.sql import *
from pyspark.sql.functions import col, asc, desc, max, sum, abs, length

In [2]:
# Criando o SparkSessiso
spark = (SparkSession.builatader
        .appName("Map")
        .getOrCreate())

In [65]:
data = spark.read.csv("/home/jovyan/work/all_stocks_5yr.csv", header=True, inferSchema=True)

data.printSchema()

root
 |-- date: string (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- Name: string (nullable = true)



In [67]:
# Retorna os registros nulos para os campos idedntificados
dataNull = data.filter(col("open").isNull() | col("high").isNull() | col("low").isNull() | col("close").isNull() | col("date").isNull() | col("volume").isNull() | col("Name").isNull())

dataNull.show(100)

+----------+----+-----+-----+-------+-------+----+
|      date|open| high|  low|  close| volume|Name|
+----------+----+-----+-----+-------+-------+----+
|2017-07-26|null| null| null|69.0842|      3| BHF|
|2015-07-17|null|88.76|88.24|  88.72|2056819| DHR|
|2016-01-12|null| null| null|  88.55|      0| DHR|
|2015-07-17|null|48.49|47.85|  47.92|1246786|  ES|
|2016-07-01|null| null| null|  49.54|      0| FTV|
|2015-07-17|null|47.31|46.83|  46.99|1229513|   O|
|2016-01-12|null| null| null|  52.43|      0|   O|
|2015-06-09|null| null| null| 526.09|  12135|REGN|
|2016-04-07|null| null| null|  41.56|      0|  UA|
|2015-05-12|null| null| null| 124.08| 569747|VRTX|
|2015-06-26|null| null| null|   61.9|    100| WRK|
+----------+----+-----+-----+-------+-------+----+



In [81]:
# Atribui 0 (zero) aos valores nulos
data = data.na.fill({"open": 0, "high": 0, "low": 0})

In [85]:
data.groupBy("open").count().orderBy(asc("open")).show(10)

+----+-----+
| low|count|
+----+-----+
| 0.0|    8|
| 1.5|    1|
|1.53|    1|
|1.56|    1|
|1.61|    1|
|1.62|    1|
|1.63|    1|
|1.65|    4|
|1.66|    4|
|1.68|    3|
+----+-----+
only showing top 10 rows



In [86]:
# Questão 1 - Total de registros
data.count()

619040

In [87]:
data.show(10)

+----------+-----+-----+-----+-----+--------+----+
|      date| open| high|  low|close|  volume|Name|
+----------+-----+-----+-----+-----+--------+----+
|2013-02-08|15.07|15.12|14.63|14.75| 8407500| AAL|
|2013-02-11|14.89|15.01|14.26|14.46| 8882000| AAL|
|2013-02-12|14.45|14.51| 14.1|14.27| 8126000| AAL|
|2013-02-13| 14.3|14.94|14.25|14.66|10259500| AAL|
|2013-02-14|14.94|14.96|13.16|13.99|31879900| AAL|
|2013-02-15|13.93|14.61|13.93| 14.5|15628000| AAL|
|2013-02-19|14.33|14.56|14.08|14.26|11354400| AAL|
|2013-02-20|14.17|14.26|13.15|13.33|14725200| AAL|
|2013-02-21|13.62|13.95| 12.9|13.37|11922100| AAL|
|2013-02-22|13.57| 13.6|13.21|13.57| 6071400| AAL|
+----------+-----+-----+-----+-----+--------+----+
only showing top 10 rows



In [93]:
# Questão 2 - Quantos registros há na planilha para a ação da Apple (AAPL)
data.select("Name").where(data.Name=="AAPL").count()

1259

In [95]:
# Questão 3 - Quantas empresas distintas têm registro nessa planilha
data.createOrReplaceTempView("df")
spark.sql('Select count(distinct Name) from df').show()

+--------------------+
|count(DISTINCT Name)|
+--------------------+
|                 505|
+--------------------+



In [96]:
data.printSchema()

root
 |-- date: string (nullable = true)
 |-- open: double (nullable = false)
 |-- high: double (nullable = false)
 |-- low: double (nullable = false)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- Name: string (nullable = true)



In [97]:
data.show(10)

+----------+-----+-----+-----+-----+--------+----+
|      date| open| high|  low|close|  volume|Name|
+----------+-----+-----+-----+-----+--------+----+
|2013-02-08|15.07|15.12|14.63|14.75| 8407500| AAL|
|2013-02-11|14.89|15.01|14.26|14.46| 8882000| AAL|
|2013-02-12|14.45|14.51| 14.1|14.27| 8126000| AAL|
|2013-02-13| 14.3|14.94|14.25|14.66|10259500| AAL|
|2013-02-14|14.94|14.96|13.16|13.99|31879900| AAL|
|2013-02-15|13.93|14.61|13.93| 14.5|15628000| AAL|
|2013-02-19|14.33|14.56|14.08|14.26|11354400| AAL|
|2013-02-20|14.17|14.26|13.15|13.33|14725200| AAL|
|2013-02-21|13.62|13.95| 12.9|13.37|11922100| AAL|
|2013-02-22|13.57| 13.6|13.21|13.57| 6071400| AAL|
+----------+-----+-----+-----+-----+--------+----+
only showing top 10 rows



In [178]:
# Questão  4 -
f"{(data.filter(col('close') > col('open')).count() / data.count()) * 100:.2f}%"

'51.53%'

In [99]:
lower = spark.sql('Select count(*) as qtd from df where close <= open')
lower.show()

+------+
|   qtd|
+------+
|300059|
+------+



In [101]:
# Questão 5 - Qual o maior valor das ações da Apple (AAPL) na história
spark.sql('Select max(open), max(high), max(low), max(close) from df where Name = "AAPL"').show()

+---------+---------+--------+----------+
|max(open)|max(high)|max(low)|max(close)|
+---------+---------+--------+----------+
|   179.37|    180.1|  178.25|    179.26|
+---------+---------+--------+----------+



In [112]:
# Questão 6 - Qual a ação tem maior volatilidade? Uma forma é medir o desvio-padrão do preço de fechamento de cada ação e considerar 
# a ação de maior desvio-padrão 
from pyspark.sql import functions
from pyspark.sql import Window


dataPartition = Window.partitionBy("Name")

dataDesvio = data.withColumn("desvio-padrao", functions.stddev("close").over(dataPartition))

dataDesvio.show()

+----------+-------+-------+-------+-------+---------+----+------------------+
|      date|   open|   high|    low|  close|   volume|Name|     desvio-padrao|
+----------+-------+-------+-------+-------+---------+----+------------------+
|2013-02-08|67.7142|68.4014|66.8928|67.8542|158168416|AAPL|30.556811676964696|
|2013-02-11|68.0714|69.2771|67.6071|68.5614|129029425|AAPL|30.556811676964696|
|2013-02-12|68.5014|68.9114|66.8205|66.8428|151829363|AAPL|30.556811676964696|
|2013-02-13|66.7442|67.6628|66.1742|66.7156|118721995|AAPL|30.556811676964696|
|2013-02-14|66.3599|67.3771|66.2885|66.6556| 88809154|AAPL|30.556811676964696|
|2013-02-15|66.9785|67.1656|65.7028|65.7371| 97924631|AAPL|30.556811676964696|
|2013-02-19|65.8714|66.1042|64.8356|65.7128|108854046|AAPL|30.556811676964696|
|2013-02-20|65.3842|65.3842|64.1142|64.1214|118891367|AAPL|30.556811676964696|
|2013-02-21|63.7142|64.1671|63.2599|63.7228|111596821|AAPL|30.556811676964696|
|2013-02-22|64.1785|64.5142|63.7999|64.4014| 8258382

In [117]:
dataDesvio.select("Name","desvio-padrao") \
          .orderBy(desc("desvio-padrao")) \
          .limit(1) \
          .show()

+----+----------------+
|Name|   desvio-padrao|
+----+----------------+
|PCLN|320.533473018748|
+----+----------------+



In [129]:
# Questão 7 - Qual o dia com maior volume de negociação da bolsa
data_sum = data.groupBy("date").agg({"volume": "sum"})

data_sum.orderBy(desc("sum(volume)")).limit(1).show()

+----------+-----------+
|      date|sum(volume)|
+----------+-----------+
|2015-08-24| 4607945196|
+----------+-----------+



In [130]:
# Questão 8 - Qual a ação mais negociada na bolsa, em volume de transações
data_max = data.groupBy("Name").agg({"volume": "sum"})

data_max.orderBy(desc("sum(volume)")).limit(1).show()

+----+------------+
|Name| sum(volume)|
+----+------------+
| BAC|117884953591|
+----+------------+



In [184]:
# Questão 9 - Quantas ações começam com a letra "A"
spark.sql('Select count(distinct Name) as qtd from df where upper(Name) like "A%"').show()

+---+
|qtd|
+---+
| 59|
+---+



In [180]:
# Questão 10 - Com qual frequência o preço mais alto do dia da ação também é o preço de fechamento
f"{(data.filter(col('high') == col('close')).count() / data.count()) * 100:.2f}%"

'1.20%'

In [146]:
# Questão 11 - Em qual dia a ação da Apple mais subiu entre a abertura e o fechamento, de forma absoluta
data_variacao = data.where(data.Name=="AAPL") 
data_variacao = data_variacao.withColumn("variacao", abs(col("close") - col("open")))

data_variacao.show()

+----------+-------+-------+-------+-------+---------+----+--------------------+
|      date|   open|   high|    low|  close|   volume|Name|            variacao|
+----------+-------+-------+-------+-------+---------+----+--------------------+
|2013-02-08|67.7142|68.4014|66.8928|67.8542|158168416|AAPL| 0.14000000000000057|
|2013-02-11|68.0714|69.2771|67.6071|68.5614|129029425|AAPL|  0.4900000000000091|
|2013-02-12|68.5014|68.9114|66.8205|66.8428|151829363|AAPL|   1.658600000000007|
|2013-02-13|66.7442|67.6628|66.1742|66.7156|118721995|AAPL|0.028600000000011505|
|2013-02-14|66.3599|67.3771|66.2885|66.6556| 88809154|AAPL| 0.29570000000001073|
|2013-02-15|66.9785|67.1656|65.7028|65.7371| 97924631|AAPL|  1.2413999999999987|
|2013-02-19|65.8714|66.1042|64.8356|65.7128|108854046|AAPL| 0.15859999999999275|
|2013-02-20|65.3842|65.3842|64.1142|64.1214|118891367|AAPL|  1.2628000000000128|
|2013-02-21|63.7142|64.1671|63.2599|63.7228|111596821|AAPL|0.008600000000001273|
|2013-02-22|64.1785|64.5142|

In [147]:
data_variacao = data_variacao.groupBy("date").agg({"variacao": "max"})

data_variacao.orderBy(desc("max(variacao)")).limit(1).show()

+----------+-------------+
|      date|max(variacao)|
+----------+-------------+
|2015-08-24|         8.25|
+----------+-------------+



In [162]:
# Questão 12 - Em média, qual o volume diário de transações das ações da AAPL
from pyspark.sql.types import DecimalType

data_media = spark.sql('Select avg(volume) as media_volume from df where upper(Name) like "AAPL"')
data_media.withColumn("media_volume", data_media.media_volume.cast(DecimalType(18, 2))).show()

+------------+
|media_volume|
+------------+
| 54047899.74|
+------------+



In [175]:
# Questão 13 - Quantas ações tem 1, 2, 3, 4, 5 caracteres em seu nome, respectivamente
data_name = spark.sql('Select distinct Name from df')
data_name = data_name.withColumn('tamanho', length(data_name.Name))
data_name.groupBy("tamanho").count().orderBy("tamanho").show()

+-------+-----+
|tamanho|count|
+-------+-----+
|      1|   10|
|      2|   50|
|      3|  323|
|      4|  117|
|      5|    5|
+-------+-----+



In [176]:
# Questão 14 - Qual a ação menos negociada na bolsa, em volume de transações
data_min = data.groupBy("Name").agg({"volume": "sum"})

data_min.orderBy(asc("sum(volume)")).limit(1).show()

+----+-----------+
|Name|sum(volume)|
+----+-----------+
|APTV|   92947779|
+----+-----------+



In [181]:
# Questão 15 - Com qual frequencia o preço do fechamento é também o mais alto do dia
f"{(data.filter(col('close') == col('high')).count() / data.count()) * 100:.2f}%"

'1.20%'